In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24750237
paper_name = 'bleackley_anderson_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/BPTI total resistance data -NA.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5153 x 8


In [7]:
original_data.head()

,plate,row,column,z score,difference,gene,Unnamed: 6,Unnamed: 7
0,8,C,3,-5.757623,-0.2343,YOL050C,NaN,NaN
1,7,C,10,-3.748354,-0.2193,YOR320C,NaN,0.124624
2,33,E,5,-5.341064,-0.1260,YHL024W,NaN,0.040284
3,7,C,12,-2.423768,-0.1179,YOR322C,NaN,NaN
4,7,C,9,-2.282688,-0.1071,YOR318C,NaN,0.205192


In [8]:
original_data['orf'] = original_data['gene'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [plate, row, column, z score, difference, gene, Unnamed: 6, Unnamed: 7, orf]
Index: []


In [15]:
original_data['data'] = -original_data['difference']

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(4944, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [567]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,567
data_type,value
orf,
YAL002W,-0.0344
YAL004W,-0.1456
YAL005C,-0.1359
YAL007C,-0.1553
YAL008W,-0.1492


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,567
,data_type,value
gene_id,orf,
2,YAL002W,-0.0344
1863,YAL004W,-0.1456
4,YAL005C,-0.1359
5,YAL007C,-0.1553
6,YAL008W,-0.1492


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          567          
data_type         value    valuez
gene_id orf                      
2       YAL002W -0.0344  2.258945
1863    YAL004W -0.1456 -0.539173
4       YAL005C -0.1359 -0.295093
5       YAL007C -0.1553 -0.783253
6       YAL008W -0.1492 -0.629759

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24750237...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.20s/it]

Updating the data_modified_on field...
